# Using SQL with Pandas DataFrames

This notebook demonstrates how to use SQL queries directly on Pandas DataFrames using the `pandasql` library. This approach allows you to leverage your SQL knowledge while working with Pandas DataFrames in memory.

## Overview

The `pandasql` library lets you query Pandas DataFrames using SQLite syntax. This is particularly useful when:
- You're more comfortable with SQL than Pandas operations
- You need to perform complex joins or aggregations that are more intuitive in SQL
- You're prototyping queries that will eventually run in a database environment

## Setup and Environment

In [ ]:
# Install required libraries if not already installed
!pip install pandas pandasql matplotlib

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandasql import sqldf
import os

# Configure matplotlib for better visualization in notebook
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
# Set up a function to run SQL queries
def run_query(q):
    """Execute an SQL query against pandas DataFrames"""
    return sqldf(q, globals())

## Loading Sample Data

Let's load our sample data from the CSV file and create a few DataFrames to work with.

In [ ]:
# Define path to the data file
data_path = '../data/sample_data.csv'

# Check if the file exists
if not os.path.exists(data_path):
    # If sample_data.csv doesn't exist, create sample data
    print(f"Sample data file not found at {data_path}. Creating sample data instead.")
    
    # Create sample sales data
    np.random.seed(42)
    dates = pd.date_range('2023-01-01', '2023-12-31', freq='D')
    
    sales_data = pd.DataFrame({
        'date': np.random.choice(dates, 1000),
        'product_id': np.random.randint(1, 11, 1000),
        'quantity': np.random.randint(1, 50, 1000),
        'unit_price': np.random.uniform(10, 1000, 1000).round(2),
        'customer_id': np.random.randint(1, 101, 1000)
    })
    
    # Calculate total price
    sales_data['total_price'] = (sales_data['quantity'] * sales_data['unit_price']).round(2)
    
    # Create products data
    products = pd.DataFrame({
        'product_id': range(1, 11),
        'product_name': [f'Product {i}' for i in range(1, 11)],
        'category': np.random.choice(['Electronics', 'Clothing', 'Food', 'Books'], 10)
    })
    
    # Create customers data
    customers = pd.DataFrame({
        'customer_id': range(1, 101),
        'customer_name': [f'Customer {i}' for i in range(1, 101)],
        'region': np.random.choice(['North', 'South', 'East', 'West'], 100)
    })
else:
    # If the file exists, load it
    print(f"Loading sample data from {data_path}")
    sales_data = pd.read_csv(data_path)

In [ ]:
# Display the first few rows of the sales data
print("Sales Data Sample:")
sales_data.head()

In [ ]:
# If we created sample data, show products and customers
if 'products' in locals():
    print("\nProducts Sample:")
    display(products.head())
    
    print("\nCustomers Sample:")
    display(customers.head())

## Basic SQL Queries with PandasSQL

Now, let's demonstrate how to use SQL to query our Pandas DataFrames.

In [ ]:
# Simple SELECT query
query = """
SELECT *
FROM sales_data
LIMIT 5
"""

result = run_query(query)
result

In [ ]:
# Filtering data with WHERE clause
query = """
SELECT *
FROM sales_data
WHERE quantity > 40
ORDER BY quantity DESC
LIMIT 10
"""

run_query(query)

In [ ]:
# Aggregation with GROUP BY
query = """
SELECT 
    product_id,
    COUNT(*) as num_transactions,
    SUM(quantity) as total_quantity,
    SUM(total_price) as total_revenue,
    AVG(unit_price) as avg_unit_price
FROM sales_data
GROUP BY product_id
ORDER BY total_revenue DESC
"""

product_summary = run_query(query)
product_summary

## Date-based Analysis

SQL provides powerful functions for working with dates. Let's use them to analyze our sales data by time periods.

In [ ]:
# Make sure date is in the correct format
if not pd.api.types.is_datetime64_any_dtype(sales_data['date']):
    sales_data['date'] = pd.to_datetime(sales_data['date'])

In [ ]:
# Monthly sales analysis
query = """
SELECT 
    strftime('%Y-%m', date) as month,
    COUNT(*) as num_transactions,
    SUM(total_price) as monthly_revenue
FROM sales_data
GROUP BY strftime('%Y-%m', date)
ORDER BY month
"""

monthly_sales = run_query(query)
monthly_sales

In [ ]:
# Plot monthly revenue
plt.figure(figsize=(12, 6))
plt.bar(monthly_sales['month'], monthly_sales['monthly_revenue'])
plt.title('Monthly Revenue')
plt.xlabel('Month')
plt.ylabel('Revenue')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Joining DataFrames with SQL

One of the strengths of SQL is the ability to join multiple tables. Let's demonstrate how to join our sales data with product and customer information.

In [ ]:
# If we don't have products and customers dataframes from CSV, use the ones created earlier
# If not available, create them now
if 'products' not in locals() and 'customers' not in locals():
    # Create products data
    products = pd.DataFrame({
        'product_id': range(1, 11),
        'product_name': [f'Product {i}' for i in range(1, 11)],
        'category': np.random.choice(['Electronics', 'Clothing', 'Food', 'Books'], 10)
    })
    
    # Create customers data
    customers = pd.DataFrame({
        'customer_id': range(1, 101),
        'customer_name': [f'Customer {i}' for i in range(1, 101)],
        'region': np.random.choice(['North', 'South', 'East', 'West'], 100)
    })

In [ ]:
# Join sales data with product information
query = """
SELECT 
    s.date,
    p.product_name,
    p.category,
    s.quantity,
    s.unit_price,
    s.total_price
FROM sales_data s
JOIN products p ON s.product_id = p.product_id
LIMIT 10
"""

sales_with_products = run_query(query)
sales_with_products

In [ ]:
# Sales by product category
query = """
SELECT 
    p.category,
    COUNT(*) as num_transactions,
    SUM(s.total_price) as total_revenue,
    AVG(s.total_price) as avg_transaction_value
FROM sales_data sJOIN products p ON s.product_id = p.product_id
GROUP BY p.category
ORDER BY total_revenue DESC
"""

category_sales = run_query(query)
category_sales

In [ ]:
# Visualize sales by category
plt.figure(figsize=(10, 6))
plt.bar(category_sales['category'], category_sales['total_revenue'])
plt.title('Revenue by Product Category')
plt.xlabel('Category')
plt.ylabel('Total Revenue')
plt.tight_layout()
plt.show()

In [ ]:
# Multi-table join with sales, products, and customers
query = """
SELECT 
    c.region,
    p.category,
    COUNT(*) as num_transactions,
    SUM(s.quantity) as total_quantity,
    SUM(s.total_price) as total_revenue,
    AVG(s.total_price) as avg_transaction_value
FROM sales_data s
JOIN products p ON s.product_id = p.product_id
JOIN customers c ON s.customer_id = c.customer_id
GROUP BY c.region, p.category
ORDER BY total_revenue DESC
"""

regional_category_sales = run_query(query)
regional_category_sales

In [ ]:
# Visualize regional category sales
plt.figure(figsize=(12, 6))
regional_category_pivot = regional_category_sales.pivot(index='region', columns='category', values='total_revenue').fillna(0)
regional_category_pivot.plot(kind='bar', stacked=True)
plt.title('Revenue by Region and Category')
plt.xlabel('Region')
plt.ylabel('Total Revenue')
plt.legend(title='Category')
plt.tight_layout()
plt.show()

## Conclusion

This notebook demonstrated how to use SQL queries with Pandas DataFrames using the pandasql library. Key points:

1. **Flexibility**: We can use familiar SQL syntax to query Pandas DataFrames
2. **Power**: Complex joins, aggregations, and date operations are straightforward
3. **Visualization**: Results can be easily visualized using matplotlib
4. **Prototyping**: Perfect for testing SQL queries before implementing in a database

Limitations to consider:
- Performance may not match native Pandas operations for large datasets
- Uses SQLite syntax specifically
- All data must fit in memory

This approach bridges the gap between SQL-based database operations and Python-based data analysis, making it an excellent tool for data analysts comfortable with SQL who are working in a Python environment.